**Very simple kernels about Forex and RUB. If you like it, give me a votes, please**

### 1. Load data

In [ ]:
import datetime
import pylab 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

In [ ]:
currency_close = pd.read_csv('../input/currency_close.csv', index_col = "Date")

In [ ]:
currency_close.head()

In [ ]:
currency_close.info()

**rename the columns**

In [ ]:
currency_close.columns

In [ ]:
currency_close = currency_close.rename(columns={'AUDUSD=X' : "AUD", 'CADUSD=X' : "CAD", 'EUR=X' : "EUR",
                                                'GBPUSD=X' : "GBP", 'JPY=X' : "JPY",'NZDUSD=X' : "NZD",
                                                'RUB=X' : "RUB", 'USDCHF=X' : "CHF"})

In [ ]:
(currency_close/currency_close.iloc[0]*100).plot(figsize=(15, 10))

**Visual analysis shows that RUB had a significant attack. Need to his find and to remove**

In [ ]:
currency_close['2016-01-04':].head()

Как видно выпад случился 6 января 2016 года. Попытаюсь удалить всю строку и проверить опять за счет графика

In [ ]:
currency_close.loc['2016-01-06', :]=None
currency_close = currency_close.dropna()

In [ ]:
(currency_close/currency_close.iloc[0]*100).plot(figsize=(15, 10))

### Data preparation

In [ ]:
cur_returns = np.log(currency_close / currency_close.shift(1))

In [ ]:
cur_returns.head()

In [ ]:
cur_returns['RUB_S'] = cur_returns['RUB'].shift(-1)
cur_returns = cur_returns.dropna()
cur_returns.head()

In [ ]:
cur_returns = cur_returns.drop('RUB', axis=1)
cur_returns.head()
cur_returns.head()

### Primary analysis

In [ ]:
cur_returns.describe()

In [ ]:
cur_returns.hist(color='k', alpha=0.5, bins=50, figsize = (20,10))
plt.show()

In [ ]:
cols = cur_returns.columns
sns_plot = sns.pairplot(cur_returns[cols])
#sns_plot.savefig('pairplot.png')

**I'll look further at the correlation. First, let's look at everything, including the ruble. Then on the shifted ruble. This will be seen at the expense of hitman**

In [ ]:
cur_returns_rus = np.log(currency_close / currency_close.shift(1))

corr_matrix = cur_returns_rus.corr()
sns.heatmap(corr_matrix)

In [ ]:
corr_matrix

In [ ]:
res = stats.probplot(cur_returns.AUD, plot=plt)

In [ ]:
res = stats.probplot(cur_returns.CAD, plot=plt)

In [ ]:
res = stats.probplot(cur_returns.EUR, plot=plt)

In [ ]:
res = stats.probplot(cur_returns.GBP, plot=plt)

In [ ]:
res = stats.probplot(cur_returns.JPY, plot=plt)

In [ ]:
res = stats.probplot(cur_returns.NZD, plot=plt)

In [ ]:
res = stats.probplot(cur_returns.CHF, plot=plt)

In [ ]:
res = stats.probplot(cur_returns.RUB_S, plot=plt)

In [ ]:
for cur in cur_returns.columns:
    print(stats.shapiro(cur_returns[cur]))

**Got what I was waiting for from qq-plot. None of the distribution is not even close to normal. You can't tell that from the charts, though.**

# Training and test samples

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = cur_returns.iloc[:, :-1]
y = cur_returns.iloc[:, -1:]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
sns.distplot(y, bins=100, kde=False, color='darkviolet', hist_kws=dict(edgecolor="k", linewidth=2))
plt.title("Распределение изменения цены в день у рубля")
plt.xlabel("Изменение")
plt.ylabel("Частота")
plt.show()

# LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, \
                                Ridge, RidgeCV
from sklearn.metrics import mean_squared_error

In [ ]:
X_df, y = cur_returns.drop('RUB_S', axis=1), cur_returns['RUB_S']

In [ ]:
def rmse(y, p):
    return np.sqrt(mean_squared_error(y, p))

In [ ]:
def beatiful_coef(coefs, feature_names=X_df.columns):
    return pd.DataFrame(coefs, index=feature_names,
                       columns=['coef']).sort_values('coef',
                                                  ascending=False)

In [ ]:
train_share = 0.7

train_part_len = int(train_share * cur_returns.shape[0])

X_train  = X_df.values[:train_part_len, :]
X_valid = X_df.values[train_part_len:, :]
y_train  = y.values[:train_part_len]
y_valid = y.values[train_part_len:]

In [ ]:
ols = LinearRegression(normalize=True)

In [ ]:
ols.fit(X_train, y_train)

In [ ]:
y_ols_pred = ols.predict(X_valid)

In [ ]:
rmse(y_valid, y_ols_pred)

In [ ]:
beatiful_coef(ols.coef_)

In [ ]:
ridge = Ridge(random_state=17, normalize=True)

In [ ]:
ridge.fit(X_train, y_train)

In [ ]:
beatiful_coef(ridge.coef_)

In [ ]:
ridge_valid_pred = ridge.predict(X_valid)

In [ ]:
rmse(y_valid, ridge_valid_pred)

In [ ]:
alphas = np.logspace(-4, 4, 100)

In [ ]:
ridge_cv = RidgeCV(alphas=alphas,
                   cv=5, normalize=True).fit(X_train, y_train)

In [ ]:
ridge_cv.alpha_

In [ ]:
best_ridge_valid_pred = ridge_cv.predict(X_valid)

In [ ]:
rmse(y_valid, best_ridge_valid_pred)

In [ ]:
lasso = LassoCV(alphas=alphas, cv=5, normalize=True).fit(X_train, y_train)

In [ ]:
lasso.alpha_

In [ ]:
best_lasso_valid_pred = lasso.predict(X_valid)

In [ ]:
rmse(y_valid, best_lasso_valid_pred)

In [ ]:
beatiful_coef(lasso.coef_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
beatiful_coef(ridge_cv.coef_)

# RandomForestRegressor

In [ ]:
forest = RandomForestRegressor(n_estimators=100, random_state=17)

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
forest_valid_pred = forest.predict(X_valid)

In [ ]:
rmse(y_valid, forest_valid_pred)

In [ ]:
beatiful_coef(forest.feature_importances_)

**Using Regression and RandomForestRegressor features in the above models turned out to by highly ineffective.**